In [1]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import psycopg2

In [2]:
transport_emp_df = pd.read_csv("CSVCleanup/Cleaned_Transportation_Employment.csv")
transport_emp_df

,Unnamed: 0,Index,Date,Pipline,Water,Rail,Air,Transit_and_Ground_Passenger_Transportation,Truck,Unemployment_Rate,Labor_Force_Participation_Rate,Unemployed
0,696,696,1/1/2005 0:00,38100.0,55500.0,191000.0,505200.0,407900.0,1348500.0,0.053,0.658,7784000.0
1,697,697,2/1/2005 0:00,37700.0,55100.0,191400.0,503400.0,407700.0,1349000.0,0.054,0.659,7980000.0
2,698,698,3/1/2005 0:00,37600.0,56700.0,191900.0,504200.0,409200.0,1359900.0,0.052,0.659,7737000.0
3,699,699,4/1/2005 0:00,37800.0,59400.0,193300.0,507300.0,413700.0,1377300.0,0.052,0.661,7672000.0
4,700,700,5/1/2005 0:00,37500.0,61100.0,193900.0,507800.0,415300.0,1391700.0,0.051,0.661,7651000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
182,879,736,5/1/2008 0:00,41500.0,67200.0,196600.0,498400.0,449100.0,1405100.0,0.054,0.661,8395000.0
183,880,735,4/1/2008 0:00,40400.0,65500.0,195900.0,499700.0,448100.0,1394400.0,0.050,0.659,7637000.0
184,881,734,3/1/2008 0:00,40500.0,62800.0,194600.0,502500.0,440800.0,1389800.0,0.051,0.661,7822000.0
185,882,733,2/1/2008 0:00,40400.0,63200.0,194300.0,501800.0,439900.0,1377300.0,0.049,0.660,7497000.0


In [3]:
transport_emp_df = transport_emp_df.drop(['Unnamed: 0'], axis = 1)

In [4]:
transport_emp_df.dtypes

Index                                            int64
Date                                            object
Pipline                                        float64
Water                                          float64
Rail                                           float64
Air                                            float64
Transit_and_Ground_Passenger_Transportation    float64
Truck                                          float64
Unemployment_Rate                              float64
Labor_Force_Participation_Rate                 float64
Unemployed                                     float64
dtype: object

In [5]:
transport_emp_df['Date'] = pd.to_datetime(transport_emp_df['Date'])
transport_emp_df['Date']

0     2005-01-01
1     2005-02-01
2     2005-03-01
3     2005-04-01
4     2005-05-01
         ...    
182   2008-05-01
183   2008-04-01
184   2008-03-01
185   2008-02-01
186   2008-01-01
Name: Date, Length: 187, dtype: datetime64[ns]

In [6]:
transport_emp_df.dtypes

Index                                                   int64
Date                                           datetime64[ns]
Pipline                                               float64
Water                                                 float64
Rail                                                  float64
Air                                                   float64
Transit_and_Ground_Passenger_Transportation           float64
Truck                                                 float64
Unemployment_Rate                                     float64
Labor_Force_Participation_Rate                        float64
Unemployed                                            float64
dtype: object

In [7]:
transport_emp_df.loc[transport_emp_df['Date']>='2010-1-1']

,Index,Date,Pipline,Water,Rail,Air,Transit_and_Ground_Passenger_Transportation,Truck,Unemployment_Rate,Labor_Force_Participation_Rate,Unemployed
36,882,2020-07-01,49700.0,58100.0,142000.0,398600.0,271400.0,1457100.0,0.102,0.614,16338000.0
37,881,2020-06-01,50000.0,57200.0,144600.0,382200.0,307000.0,1449600.0,0.111,0.615,17750000.0
38,880,2020-05-01,50400.0,57100.0,148000.0,385300.0,328900.0,1430000.0,0.133,0.608,20985000.0
39,879,2020-04-01,50500.0,57900.0,154900.0,432700.0,331300.0,1414600.0,0.147,0.602,23078000.0
40,878,2020-03-01,51300.0,62000.0,159200.0,510700.0,515300.0,1499400.0,0.044,0.627,7140000.0
...,...,...,...,...,...,...,...,...,...,...,...
158,760,2010-05-01,41700.0,62500.0,180300.0,460500.0,445700.0,1238700.0,0.096,0.649,14849000.0
159,759,2010-04-01,42400.0,60500.0,179700.0,461300.0,443400.0,1220700.0,0.099,0.652,15325000.0
160,758,2010-03-01,42400.0,60300.0,178300.0,461700.0,442800.0,1208300.0,0.099,0.649,15202000.0
161,757,2010-02-01,43500.0,58100.0,177300.0,461500.0,435800.0,1200300.0,0.098,0.649,15113000.0


In [8]:
highway_df = pd.read_csv("CSVCleanUp/Cleaned_Highway_Vehicle_Miles_Traveled.csv")
highway_df.head()

,Unnamed: 0,Index,Date,HVMT_All_Systems,HVMT_Total_Rural,HVMT_Other_Rural,HVMT_Rural_Other_Arterial,HVMT_Rural_Interstate
0,740,875,12/1/2019 0:00,2.740000e+11,8.039500e+10,2.748300e+10,3.125700e+10,2.165400e+10
1,741,874,11/1/2019 0:00,2.600000e+11,7.756300e+10,2.680900e+10,3.030200e+10,2.045200e+10
2,742,873,10/1/2019 0:00,2.840000e+11,8.676500e+10,3.040200e+10,3.395700e+10,2.240600e+10
3,743,872,9/1/2019 0:00,2.720000e+11,8.364200e+10,2.924800e+10,3.291200e+10,2.148200e+10
4,744,871,8/1/2019 0:00,2.870000e+11,9.078700e+10,3.138000e+10,3.521500e+10,2.419200e+10


In [9]:
highway_df = highway_df.drop(['Unnamed: 0'], axis = 1)

In [10]:
highway_df['Date'] = pd.to_datetime(highway_df['Date'])
highway_df['Date']

0    2019-12-01
1    2019-11-01
2    2019-10-01
3    2019-09-01
4    2019-08-01
5    2019-07-01
6    2019-06-01
7    2019-05-01
8    2019-04-01
9    2019-03-01
10   2019-02-01
11   2019-01-01
12   2018-12-01
13   2018-11-01
14   2018-10-01
15   2018-09-01
16   2018-08-01
17   2018-07-01
18   2018-06-01
19   2018-05-01
20   2018-04-01
21   2018-03-01
22   2018-02-01
23   2018-01-01
Name: Date, dtype: datetime64[ns]

In [11]:
highway_df.loc[highway_df['Date']>='2018-1-1']

,Index,Date,HVMT_All_Systems,HVMT_Total_Rural,HVMT_Other_Rural,HVMT_Rural_Other_Arterial,HVMT_Rural_Interstate
0,875,2019-12-01,2.740000e+11,8.039500e+10,2.748300e+10,3.125700e+10,2.165400e+10
1,874,2019-11-01,2.600000e+11,7.756300e+10,2.680900e+10,3.030200e+10,2.045200e+10
2,873,2019-10-01,2.840000e+11,8.676500e+10,3.040200e+10,3.395700e+10,2.240600e+10
3,872,2019-09-01,2.720000e+11,8.364200e+10,2.924800e+10,3.291200e+10,2.148200e+10
4,871,2019-08-01,2.870000e+11,9.078700e+10,3.138000e+10,3.521500e+10,2.419200e+10
5,870,2019-07-01,2.960000e+11,9.481500e+10,3.273200e+10,3.666600e+10,2.541700e+10
6,869,2019-06-01,2.810000e+11,8.738400e+10,3.052500e+10,3.385800e+10,2.300200e+10
7,868,2019-05-01,2.860000e+11,8.750000e+10,3.068700e+10,3.364500e+10,2.316700e+10
8,867,2019-04-01,2.820000e+11,8.439400e+10,3.005000e+10,3.226900e+10,2.207500e+10
9,866,2019-03-01,2.720000e+11,7.992800e+10,2.797700e+10,3.101200e+10,2.093900e+10


In [12]:
pers_spend_df = pd.read_csv("CSVCleanUp/Cleaned_Personal_Spending_on_Transportation.csv")
pers_spend_df.head()

,Unnamed: 0,Index,Date,Transportation_Services,Gas_and_Other_Engergy_Goods,Motor_Vehicles_and_Parts
0,660,660,1/1/2002 0:00,3.370000e+11,4.520000e+11,4.100000e+11
1,663,663,4/1/2002 0:00,3.340000e+11,4.570000e+11,4.090000e+11
2,666,666,7/1/2002 0:00,3.340000e+11,4.570000e+11,4.350000e+11
3,669,669,10/1/2002 0:00,3.340000e+11,4.550000e+11,4.140000e+11
4,672,672,1/1/2003 0:00,3.360000e+11,4.500000e+11,4.110000e+11


In [13]:
pers_spend_df = pers_spend_df.drop(['Unnamed: 0'], axis = 1)

In [14]:
pers_spend_df['Date'] = pd.to_datetime(pers_spend_df['Date'])
pers_spend_df['Date']

0    2002-01-01
1    2002-04-01
2    2002-07-01
3    2002-10-01
4    2003-01-01
        ...    
69   2009-01-01
70   2008-10-01
71   2008-07-01
72   2008-04-01
73   2008-01-01
Name: Date, Length: 74, dtype: datetime64[ns]

In [15]:
pers_spend_df.loc[pers_spend_df['Date']>='2010-1-1']

,Index,Date,Transportation_Services,Gas_and_Other_Engergy_Goods,Motor_Vehicles_and_Parts
24,879,2020-04-01,2.630000e+11,3.460000e+11,5.030000e+11
25,876,2020-01-01,4.150000e+11,4.210000e+11,4.960000e+11
26,873,2019-10-01,4.480000e+11,4.420000e+11,5.390000e+11
27,870,2019-07-01,4.470000e+11,4.450000e+11,5.350000e+11
28,867,2019-04-01,4.400000e+11,4.470000e+11,5.350000e+11
29,864,2019-01-01,4.350000e+11,4.470000e+11,5.210000e+11
30,861,2018-10-01,4.320000e+11,4.470000e+11,5.370000e+11
31,858,2018-07-01,4.330000e+11,4.450000e+11,5.360000e+11
32,855,2018-04-01,4.370000e+11,4.480000e+11,5.360000e+11
33,852,2018-01-01,4.410000e+11,4.480000e+11,5.310000e+11


In [16]:
rail_df = pd.read_csv("CSVCleanUp/Cleaned_Passenger_Rail.csv")
rail_df.head()

,Unnamed: 0,Index,Date,Passengers,Passenger_Miles,Total_Train_Miles,Employee_Hours_Worked,Yard_Switching_Miles
0,336,336,1/1/1975 0:00,0.0,2624696.0,4134425.0,1501878.0,0.0
1,337,337,2/1/1975 0:00,0.0,2275826.0,3600736.0,1371524.0,0.0
2,338,338,3/1/1975 0:00,0.0,2520943.0,4067192.0,1516787.0,0.0
3,339,339,4/1/1975 0:00,0.0,2478477.0,4000157.0,1583115.0,0.0
4,340,340,5/1/1975 0:00,0.0,2561119.0,4050197.0,1544540.0,0.0


In [17]:
rail_df = rail_df.drop(['Unnamed: 0'], axis = 1)

In [18]:
rail_df['Date'] = pd.to_datetime(rail_df['Date'])
rail_df['Date']

0     1975-01-01
1     1975-02-01
2     1975-03-01
3     1975-04-01
4     1975-05-01
         ...    
540   2008-05-01
541   2008-04-01
542   2008-03-01
543   2008-02-01
544   2008-01-01
Name: Date, Length: 545, dtype: datetime64[ns]

In [19]:
rail_df.loc[rail_df['Date']>='2010-1-1']

,Index,Date,Passengers,Passenger_Miles,Total_Train_Miles,Employee_Hours_Worked,Yard_Switching_Miles
396,880,2020-05-01,213983.0,69098462.0,2037397.0,2745323.0,128937.0
397,879,2020-04-01,121154.0,36018048.0,2070618.0,2745367.0,121903.0
398,878,2020-03-01,1130190.0,233377315.0,2997776.0,3073471.0,162151.0
399,877,2020-02-01,2163475.0,400433113.0,3124037.0,2945522.0,174288.0
400,876,2020-01-01,2399424.0,447564878.0,3505313.0,3147306.0,182445.0
...,...,...,...,...,...,...,...
516,760,2010-05-01,2503651.0,544408276.0,3518013.0,3451013.0,209347.0
517,759,2010-04-01,2446113.0,520013629.0,3451608.0,3478591.0,205893.0
518,758,2010-03-01,2474908.0,538675366.0,3560237.0,3629837.0,212974.0
519,757,2010-02-01,1951901.0,405905875.0,3088012.0,3122066.0,192744.0


In [20]:
air_df = pd.read_csv("CSVCleanUp/Cleaned_Airline_Traffic_Air_Carrier_Cargo.csv")
air_df.head()

,Unnamed: 0,Index,Date,Airline_Traffic_Total,Airline_Traffic_International,Airline_Traffic_Domestic,AirCarrierCargo_International_millionsOfRevenueTon-Miles,AirCarrierCargo_Domestic_millionsOfRevenueTon-Miles
0,669,669,10/1/2002 0:00,NaN,NaN,NaN,1.887142e+09,1.458666e+09
1,670,670,11/1/2002 0:00,NaN,NaN,NaN,1.778562e+09,1.296241e+09
2,671,671,12/1/2002 0:00,NaN,NaN,NaN,1.548068e+09,1.262516e+09
3,672,672,1/1/2003 0:00,NaN,NaN,NaN,1.395876e+09,1.177785e+09
4,673,673,2/1/2003 0:00,NaN,NaN,NaN,1.366617e+09,1.079561e+09


In [21]:
air_df = air_df.drop(['Unnamed: 0'], axis = 1)

In [22]:
air_df['Date'] = pd.to_datetime(air_df['Date'])
air_df['Date']

0     2002-10-01
1     2002-11-01
2     2002-12-01
3     2003-01-01
4     2003-02-01
         ...    
207   2008-05-01
208   2008-04-01
209   2008-03-01
210   2008-02-01
211   2008-01-01
Name: Date, Length: 212, dtype: datetime64[ns]

In [23]:
air_df.loc[air_df['Date']>='2010-1-1']

,Index,Date,Airline_Traffic_Total,Airline_Traffic_International,Airline_Traffic_Domestic,AirCarrierCargo_International_millionsOfRevenueTon-Miles,AirCarrierCargo_Domestic_millionsOfRevenueTon-Miles
63,880,2020-05-01,8420000.0,190000.0,8240000.0,2.228770e+09,1.583861e+09
64,879,2020-04-01,3010000.0,130000.0,2880000.0,2.025803e+09,1.447281e+09
65,878,2020-03-01,39060000.0,4650000.0,34410000.0,2.165378e+09,1.442358e+09
66,877,2020-02-01,67810000.0,7960000.0,59850000.0,1.781338e+09,1.173055e+09
67,876,2020-01-01,70760000.0,9150000.0,61610000.0,2.041418e+09,1.322461e+09
...,...,...,...,...,...,...,...
183,760,2010-05-01,NaN,NaN,NaN,2.062541e+09,1.061358e+09
184,759,2010-04-01,NaN,NaN,NaN,1.913704e+09,1.076585e+09
185,758,2010-03-01,NaN,NaN,NaN,1.971931e+09,1.085023e+09
186,757,2010-02-01,NaN,NaN,NaN,1.638815e+09,9.260431e+08
